In [103]:
%pip install python-dotenv google-genai

Note: you may need to restart the kernel to use updated packages.


In [104]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access your API key
api_key = os.getenv("GEMINI_API_KEY")

from google import genai

client = genai.Client(api_key=api_key)


In [ ]:
from pdb import set_trace

prices = {
        "TATA": {
            "HARRIER": {
                "DIESEL": {
                    "AUTOMATIC": {
                        "PUREXAT": 2303000,
                        "ADVENTUREXAT": 1957000,
                        "FEARLESSXAT": 2845000
                    },
                    "MANUAL": {
                        "SMART": 1747000,
                        "PUREX": 2116000,
                        "ADVENTUREX": 2234000,
                        "FEARLESSX": 2641000
                    }
                },
                "ELECTRIC": {
                    "AUTOMATIC": {
                        "ADVENTURE65": 2326000,
                        "FEARLESSPLUS65": 2592000,
                        "EMPOWERED75": 3256000,
                        "EMPOWEREDQWD75": 3432000
                    }
                }
            }
        }
    }

def variants(brand: str, model: str, fuel_type: str, transmission: str) -> list:
    # set_trace()
    brand = brand.upper().strip();
    model = model.upper().strip();
    fuel_type = fuel_type.upper().strip();
    transmission = transmission.upper().strip();    

    if prices[brand][model][fuel_type][transmission]:
        return list(prices[brand][model][fuel_type][transmission].keys());
    else:
        return [];

def ex_showroom_price(brand: str, model: str, fuel_type: str, transmission: str, variant: str) -> dict:
    brand = brand.upper().strip();
    model = model.upper().strip();
    fuel_type = fuel_type.upper().strip();
    transmission = transmission.upper().strip();    
    variant = variant.upper().strip();

    if prices[brand][model][fuel_type][transmission][variant]:
        return prices[brand][model][fuel_type][transmission][variant];
    else:
        return -1;

def road_tax_multiplier(state: str, ex_showroom_price: int, fuel_type: str) -> float:
    # set_trace()
    if int(ex_showroom_price) > 2500000:
        above25 = True;
    else:
        above25 = False;

    if fuel_type == "ELECTRIC" and above25:
        if state in ["DELHI", "TAMILNADU", "HYDERABAD", "MAHARASHTRA",
                      "ODISHA", "PUNJAB", "WESTBENGAL", "MEGHALAYA","BIHAR", "PUNJAB", "TELANGANA"]:
            return 0;
        if state in ["GUJARAT", "KERALA"]:
            return 0.05;
    if fuel_type == "DIESEL" :
        return 0.125;

    return 0.1;

def on_road_price(ex_showroom_price: int, road_tax_multiplier: float)->int:
    # set_trace()
    road_tax = state_development_fee = 4000;
    registration_charges = 600;
    fastag = 600;
    hypothecation_endorsement = 1500;
    other_charges = 400;
    insurance = int(int(ex_showroom_price) * 0.05);

    return int(ex_showroom_price) + road_tax + state_development_fee + registration_charges + fastag + hypothecation_endorsement + other_charges + insurance;
    

In [106]:

def function_caller(func_name, params):
    try:
        """Simple function caller that maps function names to actual functions"""
        function_map = {
            "variants": variants,
            "ex_showroom_price": ex_showroom_price,
            "road_tax_multiplier": road_tax_multiplier,
            "on_road_price": on_road_price
        }
        
        if func_name not in function_map:
            return f"Function {func_name} not found"
        
        func = function_map[func_name]

        if params.strip():
            params_list = [param.strip() for param in params.split(",")]
        else:
            params_list = []

        return func(*params_list)

    except Exception as e:
        error_msg = f"Error calling function {func_name}: {str(e)}"
        print(error_msg)
        return error_msg




In [ ]:
# from pdb import set_trace

max_iterations = 10
last_response = None
iteration = 0
iteration_response = []

system_prompt = """You are a math agent solving problems in iterations. Respond with EXACTLY ONE of these formats:
1. FUNCTION_CALL: python_function_name|input
2. FINAL_ANSWER: [dict]

input is a string of comma separated values. all json response should be valid response with fields enclosed in double quotes for strings.

1. variants(string, string, string, string) It takes brand of string type, model of string type, fuel_type of string type, transmission of string type as inputs, and returns the variants of list type.
2. ex_showroom_price(string, string, string, string) It takes brand of string type, model of string type, fuel_type of string type, transmission of string type, variant of string type as inputs, and returns the ex_showroom_price of int type.
3. road_tax_multiplier(string, int, string) It takes state of string type, ex_showroom_price of int type and fuel_type of string type as inputs, and returns the road_tax_multiplier of float type.
where python_function_name is one of the followin:
4. on_road_price(int, float) It takes ex_showroom_price of int type and road_tax_multiplier of float type as inputs, and returns the on road price as integer.
DO NOT include multiple responses. Give ONE response at a time."""

query= """Calculate the on road price of all cars with brand as tata, model as harrier, fuel type as diesel, transmission as automatic in DELHI with ex_showroom_price greater than 2500000."""
# set_trace()
while iteration < max_iterations:
    print(f"\n--- Iteration {iteration + 1} ---")
    if last_response == None:
        current_query = query
    else:
        current_query = current_query + "\n\n" + " ".join(iteration_response)
        current_query = current_query + "  What should I do next?"

    # Get model's response
    prompt = f"{system_prompt}\n\nQuery: {current_query}"
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )
    
    response_text = response.text.strip()
    print(f"LLM Response: {response_text}")

    
    if response_text.startswith("FUNCTION_CALL:"):
        response_text = response.text.strip()
        _, function_info = response_text.split(":", 1)
        func_name, params = [x.strip() for x in function_info.split("|", 1)]
        iteration_result = function_caller(func_name, params)

    # Check if it's the final answer
    elif response_text.startswith("FINAL_ANSWER:"):
        print("\n=== Agent Execution Complete ===")
        break
        

    print(f"  Result: {iteration_result}")
    last_response = iteration_result
    iteration_response.append(f"In the {iteration + 1} iteration you called {func_name} with {params} parameters, and the function returned {iteration_result}.")

    iteration += 1
    
    



--- Iteration 1 ---
LLM Response: FUNCTION_CALL: variants|tata,harrier,diesel,automatic
> /var/folders/fh/301fht2s7zd9mfp54nx_f8440000gn/T/ipykernel_15060/3878343932.py(32)variants()
     30 
     31 def variants(brand: str, model: str, fuel_type: str, transmission: str) -> list:
---> 32     set_trace()
     33     brand = brand.upper().strip();
     34     model = model.upper().strip();

  Result: ['PUREXAT', 'ADVENTUREXAT', 'FEARLESSXAT']

--- Iteration 2 ---
LLM Response: FUNCTION_CALL: ex_showroom_price|tata,harrier,diesel,automatic,PUREXAT
  Result: 2303000

--- Iteration 3 ---
LLM Response: FUNCTION_CALL: ex_showroom_price|tata,harrier,diesel,automatic,ADVENTUREXAT
  Result: 1957000

--- Iteration 4 ---
LLM Response: In the 1 iteration you called variants with tata,harrier,diesel,automatic parameters, and the function returned ['PUREXAT', 'ADVENTUREXAT', 'FEARLESSXAT']. In the 2 iteration you called ex_showroom_price with tata,harrier,diesel,automatic,PUREXAT parameters, and the